## **CSI4142 - A3: Part 1**

**Group:** 9

**Members:** 
- Jay Ghosh (300243766) 
- Alexander Azizi-Martin (300236257)

**Introduction**

#### **Dataset description**

**Dataset Name:** 

**Dataset Author:** 

**Purpose:**

### **Conclusion**

### **References**